### Demonstration of McStasScript
Here the McStasScript Python McStas API is demonstrated by creating a small simulation with an imaging calibration sample. A python function is defined in order to ease the task of adding the many materials, and a for loop is used for arranging the smaller cylinders which are embedded in a larger Aluminium cylinder. This demonstration will use widgets to control the simulation.

In [1]:
import math
%matplotlib widget

# Import McstasScript
from mcstasscript.interface import instr, plotter, functions

# Create a McStas instrument
Instr = instr.McStas_instr("calibration_sample", author = "Mads Bertelsen", origin = "ESS DMSC")

The following components are found in the work_directory / input_path:
     Union_sphere.comp
     Union_box.comp
     Single_crystal_process.comp
     Union_logger_2D_kf.comp
     Template_process.comp
     Union_conditional_standard.comp
     Union_logger_2D_space.comp
     Union_conditional_PSD.comp
     Union_master.comp
     Union_logger_2D_kf_time.comp
     Union_cylinder.comp
     Powder_process.comp
     Union_make_material.comp
     Incoherent_process.comp
     Union_logger_1D.comp
     Union_logger_3D_space.comp
     Union_logger_2DQ.comp
     Union_logger_2D_space_time.comp
These definitions will be used instead of the installed versions.


### Adding powder material definitions
Here we will be adding a few powder material definitions, and it is thus easier to create a small python function for the task. Normally the syntax would be like this for each material:

`
Al_inc = Instr.add_component("Al_inc","Incoherent_process")
Al_pow = Instr.add_component("Al_pow","Powder_process")
Al_inc.sigma = 4*0.0082 # Incoherent cross section in Barns
Al_inc.unit_cell_volume = 66.4 # Unit cell volume in AA^3
Al_pow.reflections = "\"Al.laz\"" # Data file with powder lines
Al.my_absorption = "100*4*0.231/66.4" # Inverse penetration depth in 1/m
Al.process_string = "\"Al_inc,Al_pow\"" # Make a material with aluminium incoherent and aluminium powder` 

In [2]:
def add_union_powder(name, data_name, sigma_inc, sigma_abs, unit_V, Instr):
    """
    This function adds a Union material with incoherent scattering and powder lines
    """
    material_incoherent = Instr.add_component(name + "_inc","Incoherent_process")
    material_incoherent.sigma = sigma_inc
    material_incoherent.unit_cell_volume = unit_V
    material_powder = Instr.add_component(name + "_pow","Powder_process")
    material_powder.reflections = "\"" + data_name + "\""  # Need quotes when describing a filename
    material = Instr.add_component(name,"Union_make_material")
    material.my_absorption = 100*sigma_abs/unit_V
    material.process_string = "\"" + name + "_inc," + name + "_pow" + "\""
    
# Add a number of standard powders to our instrument (datafiles included with McStas)
add_union_powder("Al", "Al.laz", 4*0.0082, 4*0.231, 66.4, Instr)
add_union_powder("Cu", "Cu.laz", 4*0.55, 4*3.78, 47.24, Instr)
add_union_powder("Ni", "Ni.laz", 4*5.2, 4*4.49, 43.76, Instr)
add_union_powder("Ti", "Ti.laz", 2*2.87, 2*6.09, 35.33, Instr)
add_union_powder("Pb", "Pb.laz", 4*0.003, 4*0.17, 121.29, Instr)
add_union_powder("Fe", "Fe.laz", 2*0.4, 2*2.56, 24.04, Instr)

### Describing a simple instrument
Now we start describing an instrument, we start with the source. We would like to control the energy and energy spread at run time, so this will be described using input parameters

In [3]:
# Since we want to change the energy and energy range at run time, we add these as instrument parameters
Instr.add_parameter("energy", value=10, comment="[meV] Energy of source")
Instr.add_parameter("delta_energy", value=8, comment="[meV] Energy spread of source")

# Add a source to the McStas instrument
src = Instr.add_component("source", "Source_div")
src.xwidth = 0.11
src.yheight = 0.11
src.focus_aw = 0.1
src.focus_ah = 0.1
src.E0 = "energy"
src.dE = "delta_energy"
src.flux = 1E13;

The source will illuminate the sample directly, but we may want to rotate the sample in its place. We define rotations around vertical and horizontal as further input parameters, and set up a few arms to act as reference points.

In [4]:
# Now we want to set a position and rotation of our sample
# The rotation should be adjustable, so we add instrument parameters for controling the rotation
Instr.add_parameter("rotation_y", value=180, comment="[deg] Rotation around vertical")
Instr.add_parameter("rotation_x", value=0, comment="[deg] Rotation around horizontal")

In [5]:
# We add arms at the sample position, and a second arm with the correct rotation
sample_pos = Instr.add_component("sample_pos", "Arm", AT=[0,0,1], AT_RELATIVE="source")
sample_arm = Instr.add_component("sample_arm", "Arm", AT=[0,0,0], AT_RELATIVE="sample_pos")
sample_arm.set_ROTATED(["rotation_x", "rotation_y", 0], RELATIVE="sample_pos")

### Adding the sample
The aim is to create a simple calibration sample which is made from cylinders of different materials embedded in a larger cylinder. To do so, we will need the McStas Union_cylinder component, so lets get a bit of help on its parameters.

In [6]:
Instr.component_help("Union_cylinder")

 ___ Help Union_cylinder ____________________________________________________________
|optional parameter|required parameter|default value|user specified value|
material_string = 0 [] // material name of this volume, defined using 
                          Union_make_material 
priority [1] // priotiry of the volume (can not be the same as another volume) 
                A high priority is on top of low. 
radius [m] // Outer radius volume in (x,z) plane
yheight [m] // Cylinder height in (y) direction
visualize = 1.0 [1] // set to 0 if you wish to hide this geometry in mcdisplay
target_index = 0
target_x = 0.0
target_y = 0.0 // Position of target to focus at
target_z = 0.0
focus_aw = 0.0 [deg] // horiz. angular dimension of a rectangular area
focus_ah = 0.0 [deg] // vert. angular dimension of a rectangular area
focus_xw = 0.0 [m] // horiz. dimension of a rectangular area
focus_xh = 0.0 [m] // vert. dimension of a rectangular area
focus_r = 0.0 [m] // focusing on circle with this radius

In [7]:
# We add the large aluminium base that contains samples of the remaining materials
base_cyl = Instr.add_component("base_cyl","Union_cylinder", RELATIVE="sample_arm")

In [8]:
# We set the parameters and confirm this looks appropriate with the print function
base_cyl.radius = 0.04
base_cyl.yheight = 0.02
base_cyl.priority = 100
base_cyl.material_string = "\"Al\"" # Select our Al material defined earlier
print(base_cyl)

COMPONENT base_cyl = Union_cylinder
  material_string = "Al" []
  priority = 100 [1]
  radius = 0.04 [m]
  yheight = 0.02 [m]
AT [0, 0, 0] RELATIVE sample_arm



Next the internal cylinders are added. We use a small python for loop to add a cylinder of each material in succession. We add an additional sample with material chosen by the user from a list of our defined materials.

In [9]:
# We define a string containing the names of the materials we wish to add
Instr.add_parameter("string", "material", value='"Pb"',
                    comment="Material choice for extra material sample",
                    options=["Cu", "Ni", "Ti", "Pb", "Fe", "Al"])

sample_strings = ["Cu", "Ni", "Ti", "Pb", "Fe", "material"]

In [10]:
# Here we write a for loop that adds a small cylinder of this material inside the large Al cylinder
angle_seperation = 36
distance_from_center = 0.03
sample_radius = 0.007
counter = 0
for sample_string in sample_strings:
    x_position = distance_from_center * math.cos(counter*angle_seperation*3.14159/180)
    x_position = round(x_position, 5) # round to 4 digits for easier printing
    z_position = distance_from_center * math.sin(counter*angle_seperation*3.14159/180)
    z_position = round(z_position, 5) # round to 4 digits for easier printing
    this_sample = Instr.add_component(sample_string + "_cyl","Union_cylinder",
                                      AT=[x_position, 0, z_position], RELATIVE="base_cyl")
    this_sample.radius = sample_radius;
    this_sample.yheight = 0.019; # yheight must be different from base_cyl
    this_sample.priority = 150 + counter; # ensure higher priority than base
    if sample_string != "material":
        this_sample.material_string = "\"" + sample_string + "\""
    else:
        # The user selectable material is a parameter, where the remaining are treated as strings.
        this_sample.material_string = sample_string
    counter = counter + 1
    
    

### Union Loggers for seeing scattering intensity in the sample
Now that the sample has been defined, a few loggers are added to investigate where neutrons are scattered within the sample.

In [11]:
# A few Union loggers are set up for display of the scattering locations
space_2D_zx = Instr.add_component("logger_space_zx_all", "Union_logger_2D_space", AT_RELATIVE="sample_pos")
space_2D_zx.filename = "\"space_zx.dat\""
space_2D_zx.D_direction_1 = "\"z\""; space_2D_zx.n1 = 1000
space_2D_zx.D1_min = -0.05; space_2D_zx.D1_max = 0.05
space_2D_zx.D_direction_2 = "\"x\""; space_2D_zx.n2 = 1000
space_2D_zx.D2_min = -0.05; space_2D_zx.D2_max = 0.05

space_2D_zy = Instr.add_component("logger_space_zy_all", "Union_logger_2D_space", AT_RELATIVE="sample_pos")
space_2D_zy.filename = "\"space_zy.dat\""
space_2D_zy.D_direction_1 = "\"z\""; space_2D_zy.n1 = 1000
space_2D_zy.D1_min = -0.05; space_2D_zy.D1_max = 0.05
space_2D_zy.D_direction_2 = "\"y\""; space_2D_zy.n2 = 1000
space_2D_zy.D2_min = -0.05; space_2D_zy.D2_max = 0.05

space_2D_zy = Instr.add_component("logger_space_xy_all", "Union_logger_2D_space", AT_RELATIVE="sample_pos")
space_2D_zy.filename = "\"space_xy.dat\""
space_2D_zy.D_direction_1 = "\"x\""; space_2D_zy.n1 = 1000
space_2D_zy.D1_min = -0.05; space_2D_zy.D1_max = 0.05
space_2D_zy.D_direction_2 = "\"y\""; space_2D_zy.n2 = 1000
space_2D_zy.D2_min = -0.05; space_2D_zy.D2_max = 0.05

### The Union master component
The Union master component is what will do the simulation of the collected calibration sample. It takes all the material information and physical volumes that we have described, and runs a simulation with full multiple scattering.

In [12]:
master = Instr.add_component("calibration_sample", "Union_master", AT_relative="sample_pos")

### McStas monitors
At the end we add a few McStas monitors to view the transmitted beam, including a PSD / energy monitor to see the Bragg edges of the different materials.

In [13]:
# Add position sensitive detector for transmission measurement
PSD = Instr.add_component("PSD", "PSD_monitor", AT=[0,0,1], RELATIVE="sample_pos") 
PSD.xwidth = 0.1; PSD.yheight = 0.1; PSD.nx = 200; PSD.ny = 200;
PSD.filename = "\"PSD.dat\""; PSD.restore_neutron = 1

In [14]:
# Adds 1D position sensitive detector for transmission measurement
PSD = Instr.add_component("PSDlin", "PSDlin_monitor", AT=[0,0,1], RELATIVE="sample_pos") 
PSD.xwidth = 0.1; PSD.yheight = 0.1; PSD.nx = 200;
PSD.filename = "\"PSDlin.dat\""; PSD.restore_neutron = 1

In [15]:
# Add energy position monitor to see Bragg edges
EPSD = Instr.add_component("EPSD", "EPSD_monitor", RELATIVE="PSD")
EPSD.xwidth = 0.1; EPSD.yheight = 0.02; EPSD.nE = 300; EPSD.nx = 200;
EPSD.filename = "\"EPSD.dat\""; EPSD.restore_neutron = 1;
EPSD.Emin = "energy - delta_energy"
EPSD.Emax = "energy + delta_energy"

### Last check 
Before running the McStas simulation we do a last check to see that the McStas instrument looks as we expect. First the components are listed with their locations and rotations, then the available parameters are shown.

In [16]:
Instr.print_components(line_length=117) # Show nice overview

Al_inc              Incoherent_process    AT      (0, 0, 0)                   ABSOLUTE           
Al_pow              Powder_process        AT      (0, 0, 0)                   ABSOLUTE           
Al                  Union_make_material   AT      (0, 0, 0)                   ABSOLUTE           
Cu_inc              Incoherent_process    AT      (0, 0, 0)                   ABSOLUTE           
Cu_pow              Powder_process        AT      (0, 0, 0)                   ABSOLUTE           
Cu                  Union_make_material   AT      (0, 0, 0)                   ABSOLUTE           
Ni_inc              Incoherent_process    AT      (0, 0, 0)                   ABSOLUTE           
Ni_pow              Powder_process        AT      (0, 0, 0)                   ABSOLUTE           
Ni                  Union_make_material   AT      (0, 0, 0)                   ABSOLUTE           
Ti_inc              Incoherent_process    AT      (0, 0, 0)                   ABSOLUTE           
Ti_pow              

### Running the simulation
The simulation can now be performed from the Jupyter Notebook using the widget interface.

In [21]:
Instr.interface()

### Getting data from the widget simulation
It is possible to grab the latest data produced by the widget simulation. Run these cells after a simulation has been performed with the above widget.

In [22]:
data = Instr.get_interface_data()
print(data)

[
McStasData: logger_space_zx_all type: 2D  I:4.46366e+09 E:1.38106e+07 N:167764, 
McStasData: logger_space_zy_all type: 2D  I:4.46366e+09 E:1.38106e+07 N:167764, 
McStasData: logger_space_xy_all type: 2D  I:4.46366e+09 E:1.38106e+07 N:167764, 
McStasData: PSD type: 2D  I:4.35753e+10 E:5.06929e+07 N:738940, 
McStasData: PSDlin type: 1D  I:4.35753e+10 E:5.06929e+07 N:738940, 
McStasData: EPSD type: 2D  I:4.62274e+09 E:1.65109e+07 N:78398]


In [23]:
if len(data) != 0:
    EPSD_data = functions.name_search("EPSD", data)
    plotter.make_plot(EPSD_data, figsize=(9,6))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Plotting data with name EPSD


### Using plotting interface without simulation interface
The plotting interface can be used for existing datasets.

In [24]:
plotter.interface(data)